# Practical work 5: matrix completion for inpainting

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Load nt_toolbox
import sys
#sys.path.append('../tp1/')
from nt_toolbox.general import rescale
from nt_toolbox.signal import load_image
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def show_image(im, title=""):
    """Display an image im (2D array-like) with a title.
    """
    plt.imshow(im, cmap='gray')
    plt.axis('off')
    plt.title(title)

## Table of contents
>I. [Low-rank matrices](#I.-Low-rank-matrices)

>II. [Image reconstruction](#II.-Image-reconstruction)

>1. [Masked image](#1.-Masked-image)
1. [Nuclear norm minimization](#2.-Nuclear-norm-minimization)

> III. [Proximal methods](#III.-Proximal-methods)
1. [Exact problem](#1.-Exact-problem)
1. [Regularized problem](#2.-Regularized-problem)

> IV. [Stability to permutation](#IV.-Stability to permutation)

> V. [Evaluation of the coherence](#Evaluation of the coherence)

## I. Low-rank matrices
>This part is aimed at demonstrating that real-world images benefit from an inherent sparsity, that is of a different kind than Fourier/Wavelet sparsity.
For this purpose, let us load and display an image.

In [ ]:
im = rescale(load_image("ryan_lalaland.bmp")) # Load the image with prescribed size
show_image(im, 'Original image')  # Display the image

>**Question 1.**
Plot together the $n$ singular values $(\sigma_i)_{1 \le i \le n}$ of the image ($\sigma_{(1)} \ge \dots \ge  \sigma_{(n)}$) and the ratio of inertia $\sum_{i=1}^m \sigma_{(i)}/\sum_{i=1}^n \sigma_{(i)}$ as a function of the ratio of singular values $\frac mn$.

>What can you say about image sparsity / rank ? How many singular values do you need to get 80% of the information?

>**Question 2.**
Given that an approximation of the image can be obtained by setting the smallest singular values to $0$, display several approximations for different ratio of singular values (ranging from $0$ to $20\%$) / inertia (ranging from $50\%$ to $100\%$).
You may need functions *trunc_r(im, ratio)* and *trunc_i(im, inertia)* that compute an approximation of the image *im* given a ratio of singular values or of inertia.

In [ ]:
print("Approximation with respect to the ratio of singular values.")
plt.figure(figsize=(10, 10))
for i, r in enumerate(np.linspace(0, 0.2, num=16)):
    plt.subplot(4, 4, i+1)
    show_image(trunc_r(im, r), 'Ratio: {0:0.2f}%'.format(100*r))

In [ ]:
print("Approximation with respect to the inerta ratio.")
plt.figure(figsize=(10, 10))
for i, r in enumerate(np.linspace(0.5, 1, num=16)):
    plt.subplot(4, 4, i+1)
    show_image(trunc_i(im, r), 'Inertia: {0:0.2f}%'.format(100*r))

## II. Image reconstruction
### 1. Masked image
>In image reconstruction (also known as image inpainting or matrix completion), we observe an image with missing values.
In practice, given an image $I \in \mathbb R^{n_1 \times n_2}$, we consider being provided with a mask $O \in \mathbb R^{n_1 \times n_2}$, such that each entry of $O$ equals $1$ if we observe a pixel and $0$ otherwise.
Therefore, if we consider that missing data is set to $0$, we observe $O \odot I$, where $\odot$ is the pointwise product.

>This is what is done in the following script, where the image is first made explicitely low-rank (by truncation of its singular values) and masked.
In this function, *ratio* is the ratio of singular values kept unscathed, and *prop* is the proportion of observed pixels.

In [ ]:
def mask(im, ratio=0.1, prop=0.2):
    im_trunc = trunc_r(im, ratio)  # Approximation by truncation
    mat_mask = np.random.binomial(1, prop, size=im.shape)  # Random mask (iid Bernoulli variables)
    return im_trunc, mat_mask

im_trunc, mat_mask = mask(im, ratio=0.2, prop=0.4)  # Approximate image and mask matrix
im_masked = im_trunc * mat_mask  # Observed image

for p, (i, t) in enumerate([(im, 'Original image'),
                           (im_trunc, 'Truncated image'),
                           (im_masked, 'Masked image')]):
    plt.subplot(1, 3, p+1)
    show_image(i, t)

### 2. Nuclear norm minimization
>Let $I \in \mathbb R ^{n_1 \times n_2}$ be the orginal image, $O \in \mathbb R ^{n_1 \times n_2}$ be mask of observed pixels and $M \colon \mathbb R ^{n_1 \times n_2} \to \mathbb R ^{n_1 \times n_2}$ be the mask operator, such that $MX = O \odot X$.

>We aim at solving the nuclear norm minimization problem:
$$
    \begin{array}{cl}
        \displaystyle{ \operatorname{minimize}_{X \in \mathbb R ^{n_1 \times n_2}} }
        & \displaystyle{ \|X\|_{S_1} } \\
        \operatorname{st}
        & \displaystyle{ MX = MI.}
    \end{array}
$$

>*cvxpy* is a user-friendly interface that helps solving such a problem (in practice, it calls solvers such as *cvxopt*).

>**Question 1.**
There are two mistakes in the following script.
Fix them and compare the image reconstructed by filling the average value and by nuclear norm minimization.
What do you feel about them?

In [ ]:
import cvxpy

def mat_compl_py(im_trunc, mat_mask):
    im_masked = im_trunc * mat_mask  # Observed image
    n1, n2 = im_masked.shape  # Image size

    X = cvxpy.Variable((n1, n2))  # Optimization variable
    obj = cvxpy.Minimize(cvxpy.norm(X, 'fro'))  # Objective function
    constraints = [cvxpy.multiply(mat_mask, X) == im_masked]  # Constraint

    prob = cvxpy.Problem(obj, constraints)  # Optimization problem
    prob.solve(solver=cvxpy.SCS)  # Solve the problem
    
    if prob.status != 'optimal':
        print('CVXPY failed to reach optimal value.')
    else:
        im_rec = np.asarray(X.value)  # Solution = reconstructed image
        
        # Average filling: fill missing pixels with mean of observed pixels
        im_fill = im_masked.copy()
        im_fill[np.where(mat_mask == 0)] = im_masked[np.where(mat_mask == 1)].max()
        
        plt.figure(figsize=(15, 5))
        for p, (i, t) in enumerate([(im_trunc, 'Truncated image'),
                                    (im_masked, 'Masked image'),
                                    (im_fill, 'Average filling'),
                                    (im_rec, 'Reconstructed image')]):
            plt.subplot(1, 4, p+1)
            show_image(i, t)
            
        return im_fill, im_rec
            
im_fill, im_rec = mat_compl_py(im_trunc, mat_mask)

**Answer:**

>**Question 2.**
Plot the histogram of recovered pixels for both reconstruction techniques. Compare with the histogram of original pixels.
What can you say?

>**Question 3.**
Create an image that is sparse and low-rank. 

>Mask the image and apply the reconstruction. What can you conclude ?

**Answer:**
sparse and low-rank images are difficult to reconstruct.
This is known from theory since matrice completion works well only when the singular vectors (of the image to be reconstructed) are incoherent with the canonical basis.
This is not the case here.

## III. Proximal methods
### 1. Exact problem
>We aim here at solving the problem of matrix completion:
$$
    \operatorname{minimize}_{X \in \mathbb R ^{n_1 \times n_2}}
    \|X\|_{S_1} + \chi_{\mathcal A}(X),
$$
where $\mathcal A = \{X \in \mathbb R^{n_1 \times n_2} : MX = MI\}$.

>** Question 1.**
Knowing that $\operatorname{prox}_{\gamma \|\cdot\|_{S_1}}$ is the soft-thresholding operator on the singular values, i.e. $\operatorname{prox}_{\gamma \|\cdot\|_{S_1}}(X) = US_\gamma V$ where $X = USV$ is the singular value decomposition of $X$ and $S_\gamma$ is the diagional matrix with entries $(\max(0, S_{kk}-\gamma))_k$, provide a function *proxS1(x, gamma)* that computes $\operatorname{prox}_{\gamma \|\cdot\|_{S_1}}(x)$.

>** Question 2.**
Show that $\operatorname{prox}_{\chi_{\mathcal A}}(X) = X - M(X - I)$.



>** Question 3.**
Define a function *DRmethod(im_masked, mat_mask, n_it=100, version=1)* that implements the Douglas-Rachford method with $f = \|\cdot\|_{S_1}$ and $g = \chi_{\mathcal A}$, and conversely with $f = \chi_{\mathcal A}$ and $g = \|\cdot\|_{S_1}$.
This function has to return an approximate solution and the sequence $(\|X_k\|_{S_1})_k$.

>Compare both versions, the behavior of the objective value and the error $\|MX-MI\|_F$.
Which one is preferable?

### 2. Regularized problem
>In this section, we decide to approximate the linear constraint with a regularization.
Therefore, we aim at solving:
$$
    \operatorname{minimize}_{X \in \mathbb R ^{n_1 \times n_2}}
    \|X\|_{S_1} + \frac{\mu}{2} \|MX - MI\|_F^2,
$$
where $\mu > 0$.

>**Question 2.**
Define a function *fista(im_masked, mat_mask, mu=1., n_it=100)*, that:
- performs an accelerated proximal gradient descent with fixed step size;
- terminates after n_it iterations;
- returns an approximate solution $X$ and the sequence of objective values $(\|X_k\|_{S_1} + \frac{\mu}{2} \|MX_k - MI\|_F^2)_k$.

>Run this function to recover the original image.
Plot the image, the objective function and compute the error $\|MX - MI\|_F$.

## IV. Stability to permutation

In this section, we propose to fix some mask $O$ and
1. to perform the low-rank recovery reconstruction of an image 'im', called 'im_rec', then 
+ to permute the columns of 'im' and call the resulting image 'im_flip'
+ to perform the low-rank recovery reconstruction of 'im_flip', called 'im_rec_flip', then 
+ to unflip the columns of 'im_rec_flip' and to compare to 'im_rec'.


To your opinion, what is the expected result to this experiment? Once your prediction made, launch the following code.

In [ ]:
# Initial image
im_trunc, mat_mask = mask(im, ratio=0.2, prop=0.4)  # Approximate image and mask matrix
im_masked = im_trunc * mat_mask  # Observed image
show_image(im_trunc, 'Original image')

In [ ]:
# Flip reconstruction + Unflipping
from toolbox.flip import low_rank_recovery_flip_test_columns

im_rec_flip_unflipped = low_rank_recovery_flip_test_columns(im_masked, mat_mask, n_it=100)
show_image(im_rec_flip_unflipped , 'Unflipped reconstructed image')

## V. Evaluation of the coherence

During Lecture \# 9, we introduced the notion of coherence for the matrix completion problem. When observed entries of a matrix $X\in \mathbb{R}^{n_1 \times n_2}$ are uniformly drawn, the coherence $\mu(X)$ is the smallest number such that
$$
\left\{
\begin{array}{ll}
    \max_{1\leq i \leq n_1} \frac{n_1}{r} \| P_{col(X)} e_i \|_2^2 &\leq \mu(X) \\
    \max_{1\leq i \leq n_2} \frac{n_2}{r} \| P_{row(X)} e_i \|_2^2 &\leq \mu(X) \\
\end{array}
\right.
$$
where $P_{col(X)}$ and $P_{row(X)}$ are the projection onto the column space and row space of $X$ (resp. subsets of $\mathbb{R}^{n_1}$ and $\mathbb{R}^{n_2}$). Set $X=UDV^T$ to be a SVD of $X$.  If $X$ is of rank $r$, one can write that 
$$
X= U^{(r)} D^{(r)} (V^{(r)})^T,
$$
where $U^{(r)} \in \mathbb{R}^{n_1\times r}$, $D^{(r)}\in \mathbb{R}^{r\times r}$ and $ V^{(r)} \in \mathbb{R}^{n_2\times r}$, and where $U^{(r)}$ and $V^{(r)}$ have orthogonal columns that contain
the left and right singular vectors.

The orthogonal projection matrix onto the column
space of $X$ is then
$$
P_{col(X)} = U^{(r)} (U^{(r)})^T \in \mathbb{R}^{n_1 \times n_1},
$$
and on the row space of $X$
$$
P_{row(X)} = V^{(r)} (V^{(r)})^T \in \mathbb{R}^{n_2 \times n_2},
$$

1. Evaluate the coherence parameter for the truncated image where the first 20\% singular values are kept.
+ Conclude on the chance of recovery using matrix completion technique for such an image.

The coherence is really high here, which in theory prescribes for matrix completion with a low number of measurements.